In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
import os
print(os.getcwd())

C:\Users\mirnalini.gunaraj\Python_Viz\path\to\myenv


In [3]:
file_path = os.path.abspath('bank_customer_data.csv')

In [4]:
if os.path.exists(file_path):
    print("File exists!")
else:
    print("File does not exist.")

File exists!


In [52]:
df = pd.read_csv(file_path)

In [53]:
df.columns

Index(['year', 'RowNumber', 'CustomerId', 'Surname', 'CreditScore',
       'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [54]:
df


,year,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,2004,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2006,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,2010,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,2005,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,2015,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2000,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,2008,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,2009,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,2010,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [55]:
df = df.fillna(0)

In [56]:
#make dataframe pipeline interactive
idf = df.interactive()

In [58]:
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=2000, end=2016, step=5, value=2000)
year_slider

IntSlider(end=2016, name='Year slider', start=2000, step=5, value=2000)

In [59]:
# Radio buttons 
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Balance', 'EstimatedSalary',],
    button_type='success'
)

In [60]:
df.Geography.value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [61]:
countries = ['France','Germany','Spain']

bal_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.Geography.isin(countries))
    ]
    .groupby(['Geography', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

# Balance Over Time

In [62]:
bal_pipeline

In [63]:
bal_plot = bal_pipeline.hvplot(x = 'year', by='Geography', y=yaxis_co2,line_width=2, title="Balance by country")
bal_plot

# Balance Over Time in Table

In [64]:
bal_table = bal_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
bal_table


In [65]:
bal_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) 
    ]
    .groupby(['Geography', 'year','EstimatedSalary'])['Balance'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [66]:
bal_vs_gdp_scatterplot_pipeline

# Scatter Plot

In [67]:
bal_vs_gdp_scatterplot = bal_vs_gdp_scatterplot_pipeline.hvplot(x='EstimatedSalary', 
                                                                y='Balance', 
                                                                by='Geography', 
                                                                size=80, 
                                                                kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
bal_vs_gdp_scatterplot 

# Bar Chart

In [75]:
y_axis_bal_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Age'], 
    button_type='success'
)

bal_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) 
    ]
    .groupby(['Geography', 'year'])[y_axis_bal_source.value].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

bal_source_bar_plot = bal_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='Geography', 
                                                     y=y_axis_bal_source.value,  # Use the correct variable here
                                                     title='Bar Chart')
bal_source_bar_plot


In [79]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Bank Transactions dashboard', 
    sidebar=[pn.pane.Markdown("# Balance and Estimated Salary Visualisation"), 
             pn.pane.Markdown("#### This dashboard shows the Balance and Estimated Salary in different dimensions and the beauty of this dashboard is that it is created using Panel and hvplot. Use the Sliders to toggle the years"), 
             pn.pane.PNG('climate_day.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, 
                           bal_plot.panel(width=700), margin=(0,25)), 
                 bal_table.panel(width=500)), 
          pn.Row(pn.Column(bal_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(y_axis_bal_source, bal_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();